<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220719.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* customer analysis
* iterator

In [91]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timezone, timedelta
import os

from functools import reduce
from collections import ChainMap, Counter, namedtuple, defaultdict

# customer analysis

In [2]:
os.listdir()

['.config',
 'listings.csv',
 'sample_submission.csv',
 'ecommerce-data.csv',
 'retail_data_response.csv',
 'shops.csv',
 'items.csv',
 'retail_data_transactions.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 '.Rhistory',
 'sales_train.csv',
 'rfm_test.csv',
 'online_retail_II.xlsx',
 'test.csv',
 'item_categories.csv',
 'sample_data']

In [4]:
df = pd.read_csv('ecommerce-data.csv', encoding = 'unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [13]:
import re

In [24]:
["_".join(re.split(r'(?=[A-Z])', a)[1:]).lower() for a in df.columns]

['invoice_no',
 'stock_code',
 'description',
 'quantity',
 'invoice_date',
 'unit_price',
 'customer_i_d',
 'country']

In [25]:
df.columns = [a.lower() for a in df.columns]

In [26]:
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [27]:
df = df[(~df.invoiceno.astype(str).str.contains('C')) & (df.country == 'United Kingdom')]
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [30]:
df['date'] = pd.to_datetime(df.invoicedate).to_numpy().astype('datetime64[D]')
df['month'] = df['date'].to_numpy().astype('datetime64[M]')
df['rev'] = df.quantity * df.unitprice
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,date,month,rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,20.34


In [33]:
df = df[['customerid', 'date', 'month', 'rev']].rename(columns = {'customerid' : 'id'})
df.head()

,id,date,month,rev
0,17850.0,2010-12-01,2010-12-01,15.30
1,17850.0,2010-12-01,2010-12-01,20.34
2,17850.0,2010-12-01,2010-12-01,22.00
3,17850.0,2010-12-01,2010-12-01,20.34
4,17850.0,2010-12-01,2010-12-01,20.34


In [34]:
df = df.groupby(['id','date','month'])['rev'].sum().reset_index()
df.head()

,id,date,month,rev
0,12346.0,2011-01-18,2011-01-01,77183.60
1,12747.0,2010-12-05,2010-12-01,358.56
2,12747.0,2010-12-13,2010-12-01,347.71
3,12747.0,2011-01-20,2011-01-01,303.04
4,12747.0,2011-03-01,2011-03-01,310.78


In [42]:
info_m = df.groupby(['id','month'])['rev'].sum().reset_index().rename(columns = {'rev':'money'})
info_m = pd.pivot_table(info_m, index = 'id', columns = 'month', values = 'money').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'month', value_name = 'money').sort_values(['id','month'],ascending = [1,0])
info_m = info_m.groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,12346.0,0.00
1,12747.0,678.00
2,12748.0,3787.09
3,12749.0,1896.13
4,12820.0,0.00


In [43]:
info_f = df.groupby(['id','month'])['rev'].count().reset_index().rename(columns = {'rev':'freq'})
info_f = pd.pivot_table(info_f, index = 'id', columns = 'month', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'month', value_name = 'freq').sort_values(['id','month'],ascending = [1,0])
info_f = info_f.groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,12346.0,0.0
1,12747.0,2.0
2,12748.0,28.0
3,12749.0,1.0
4,12820.0,0.0


In [44]:
df['date'].max()

Timestamp('2011-08-25 00:00:00')

In [46]:
date_now = datetime(2011,8,31)

In [50]:
info_r = df.groupby(['id']).agg({'date': lambda x : (date_now - x.max()).days}).reset_index().rename(columns = {'date':'recency'})
info_r.head()

,id,recency
0,12346.0,225
1,12747.0,9
2,12748.0,6
3,12749.0,30
4,12820.0,226


In [51]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info.head()

,id,recency,freq,money
0,12346.0,225,0.0,0.00
1,12747.0,9,2.0,678.00
2,12748.0,6,28.0,3787.09
3,12749.0,30,1.0,1896.13
4,12820.0,226,0.0,0.00


In [54]:
info = info.assign(r = pd.qcut(info.recency, 5, list('54321')),
            f = pd.qcut(info.freq.rank(method = 'first'), 5, list('12345')),
            m = pd.qcut(info.money.rank(method = 'first'), 5, list('12345'))
            )
info.head()

,id,recency,freq,money,r,f,m
0,12346.0,225,0.0,0.00,1,1,1
1,12747.0,9,2.0,678.00,5,4,5
2,12748.0,6,28.0,3787.09,5,5,5
3,12749.0,30,1.0,1896.13,4,3,5
4,12820.0,226,0.0,0.00,1,1,1


In [55]:
info.assign(rf = info.r.astype(str) + info.f.astype(str))

,id,recency,freq,money,r,f,m,rf
0,12346.0,225,0.0,0.00,1,1,1,11
1,12747.0,9,2.0,678.00,5,4,5,54
2,12748.0,6,28.0,3787.09,5,5,5,55
3,12749.0,30,1.0,1896.13,4,3,5,43
4,12820.0,226,0.0,0.00,1,1,1,11
...,...,...,...,...,...,...,...,...
2964,18280.0,177,0.0,0.00,1,3,3,13
2965,18281.0,80,1.0,80.82,3,4,3,34
2966,18282.0,26,1.0,100.21,4,4,3,44
2967,18283.0,48,3.0,450.72,4,5,4,45


# iterator

In [57]:
x = pd.read_csv('ecommerce-data.csv', encoding = 'unicode_escape')
x.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [62]:
['_'.join(re.split(r'(?=[A-Z])', a)[1:]).lower() for a in x.columns]


['invoice_no',
 'stock_code',
 'description',
 'quantity',
 'invoice_date',
 'unit_price',
 'customer_i_d',
 'country']

In [63]:
mat = [list('abc'), list('def'), list('hig')]
mat

[['a', 'b', 'c'], ['d', 'e', 'f'], ['h', 'i', 'g']]

In [65]:
for r, row in enumerate(mat) :
  for c, letter in enumerate(row) :
    print(f'({r}, {c}) - {letter}')

(0, 0) - a
(0, 1) - b
(0, 2) - c
(1, 0) - d
(1, 1) - e
(1, 2) - f
(2, 0) - h
(2, 1) - i
(2, 2) - g


In [66]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'age': 73,
  'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M'},
 {'age': 29,
  'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F'},
 {'age': 51,
  'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M'},
 {'age': 32,
  'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F'},
 {'age': 42, 'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F'}]

In [68]:
list_key = list(users[0].keys())

for key in list_key : 
  if key == list_key[0] :
    res = eval(f'reduce(lambda acc, cur : acc + [cur["{key}"]], users, [])')
  else :
    res = np.c_[res, eval(f'reduce(lambda acc, cur : acc + [cur["{key}"]], users, [])')]

res

array([['gregorythomas@gmail.com', 'Brett Holland', 'M', '73'],
       ['hintoncynthia@hotmail.com', 'Madison Martinez', 'F', '29'],
       ['wwagner@gmail.com', 'Michael Jenkins', 'M', '51'],
       ['daniel79@gmail.com', 'Karen Rodriguez', 'F', '32'],
       ['ujackson@gmail.com', 'Amber Rhodes', 'F', '42']], dtype='<U25')

In [72]:
a = np.random.randint(1, 10, 1000)
a[:10]

array([7, 6, 5, 4, 9, 1, 9, 1, 5, 8])

In [82]:
dict(Counter(a))

{1: 119, 2: 120, 3: 109, 4: 100, 5: 102, 6: 117, 7: 107, 8: 121, 9: 105}

In [83]:
a, b = list('123'), list('ABC')
print(a)
print(b)

['1', '2', '3']
['A', 'B', 'C']


In [84]:
ChainMap(a, b)

ChainMap(['1', '2', '3'], ['A', 'B', 'C'])

In [86]:
portfolio = [
    ('GOOG', 100, 490.1),
    ('IBM', 50, 91.1),
    ('CAT', 150, 83.44),
    ('IBM', 100, 45.23),
    ('GOOG', 75, 572.45),
    ('AA', 50, 23.15)
]
portfolio

[('GOOG', 100, 490.1),
 ('IBM', 50, 91.1),
 ('CAT', 150, 83.44),
 ('IBM', 100, 45.23),
 ('GOOG', 75, 572.45),
 ('AA', 50, 23.15)]

In [87]:
n = Counter()

for com, share, score in portfolio : 
  n[com] += share

n

Counter({'AA': 50, 'CAT': 150, 'GOOG': 175, 'IBM': 150})

In [96]:
n = defaultdict(list)

for com, share, score in portfolio : 
  n[com].append([share, score])
n

defaultdict(list,
            {'AA': [[50, 23.15]],
             'CAT': [[150, 83.44]],
             'GOOG': [[100, 490.1], [75, 572.45]],
             'IBM': [[50, 91.1], [100, 45.23]]})